<a href="https://colab.research.google.com/github/matbot/pandas_practice/blob/master/pandas_AB_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

In [28]:
ad_clicks = pd.read_csv('https://raw.githubusercontent.com/dinhanhthi/data-science-learning/master/codecademy-data-science/course-10%20Data%20Analysis%20with%20Pandas/ad_clicks.csv')
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


Basic Analysis

In [29]:
ad_clicks.groupby('utm_source').count().reset_index()

,utm_source,user_id,day,ad_click_timestamp,experimental_group
0,email,255,255,80,255
1,facebook,504,504,180,504
2,google,680,680,239,680
3,twitter,215,215,66,215


In [30]:
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.notnull()
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [31]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [32]:
clicks_pivot = clicks_by_source.pivot('utm_source', 'is_click', 'user_id')
clicks_pivot

is_click,False,True
utm_source,,
email,175,80
facebook,324,180
google,441,239
twitter,149,66


In [33]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False]) * 100
clicks_pivot

is_click,False,True,percent_clicked
utm_source,,,
email,175,80,31.372549
facebook,324,180,35.714286
google,441,239,35.147059
twitter,149,66,30.697674


A/B Test Analysis

In [34]:
ad_clicks.groupby('experimental_group').user_id.count().reset_index()

,experimental_group,user_id
0,A,827
1,B,827


In [35]:
ab_clicks = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
ab_clicks

,experimental_group,is_click,user_id
0,A,False,517
1,A,True,310
2,B,False,572
3,B,True,255


In [36]:
ab_clicks_pivot = ab_clicks.pivot('experimental_group', 'is_click', 'user_id')
ab_clicks_pivot

is_click,False,True
experimental_group,,
A,517,310
B,572,255


In [56]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']

a_clicks_pivot = a_clicks.groupby(['day', 'is_click']).user_id.count()\
  .reset_index()\
  .pivot('day', 'is_click', 'user_id')\
  .reset_index()

a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False]) * 100

a_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,38.053097
1,2 - Tuesday,76,43,36.134454
2,3 - Wednesday,86,38,30.645161
3,4 - Thursday,69,47,40.517241
4,5 - Friday,77,51,39.843750
5,6 - Saturday,73,45,38.135593
6,7 - Sunday,66,43,39.449541


In [57]:
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

b_clicks_pivot = b_clicks.groupby(['day', 'is_click']).user_id.count()\
  .reset_index()\
  .pivot('day', 'is_click', 'user_id')\
  .reset_index()

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False]) * 100

b_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,28.318584
1,2 - Tuesday,74,45,37.815126
2,3 - Wednesday,89,35,28.225806
3,4 - Thursday,87,29,25.000000
4,5 - Friday,90,38,29.687500
5,6 - Saturday,76,42,35.593220
6,7 - Sunday,75,34,31.192661
